In [1]:
import pandas as pd
import numpy as np
from joblib import dump, load

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

Helper Functions

In [2]:
def compute_metrics(y_true, y_pred):
    """Compute RMSE, MAE, and R2."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

In [3]:
def compute_stability(model, X, y, noise_level=0.01, n_trials=5):
    """
    Add Gaussian noise to numeric features and measure average relative RMSE change.
    noise_level is fraction of std-dev of each feature.
    """
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    base_rmse = np.sqrt(mean_squared_error(y, model.predict(X)))
    rel_changes = []
    
    for _ in range(n_trials):
        Xp = X.copy()
        noise = np.random.normal(0, noise_level * Xp[numeric_cols].std(), 
                                 size=Xp[numeric_cols].shape)
        Xp[numeric_cols] += noise
        rp = model.predict(Xp)
        rmse_p = np.sqrt(mean_squared_error(y, rp))
        rel_changes.append((rmse_p - base_rmse) / base_rmse)
    
    # Return average relative change (lower = more stable)
    return np.mean(rel_changes)

# Model Definition

In [4]:
models = {
    'Ridge': Ridge(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, use_label_encoder=False, eval_metric='rmse', random_state=42),
    'MLP': MLPRegressor(hidden_layer_sizes=(64, 64),    # implemented a simple MLP with sklearn for easy compatibility with the rest of the code
                        activation='relu',
                        solver='adam',
                        max_iter=200,
                        random_state=42)
}

In [5]:
datasets = load('..\data\experimental\experiment_datasets_2.joblib')  # Load datasets from joblib file

# Manual Training

In [6]:
train_df = datasets['baselines']['full']['within_sample']['within_sample']['train']
test_df = datasets['baselines']['full']['within_sample']['within_sample']['test']

train_df.shape, test_df.shape

((291, 82), (73, 82))

In [7]:
X_train = train_df.drop(columns=['totalEsg'])
y_train = train_df['totalEsg']
X_test  = test_df.drop(columns=['totalEsg'])
y_test  = test_df['totalEsg']

In [8]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((291, 81), (291,), (73, 81), (73,))

In [9]:
model = models['Ridge']  # Choose the model you want to use

In [10]:
model

Ridge(random_state=42)

In [11]:
model.fit(X_train, y_train)

l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=5.17923e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge(random_state=42)

In [12]:
# predict
y_pred = model.predict(X_test)

In [13]:
# compute metrics
rmse, mae, r2 = compute_metrics(y_test, y_pred)
stability = compute_stability(model, X_test, y_test,
                            noise_level=0.01, n_trials=5)

In [14]:
manual_results = {
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'Stability': stability
        }

In [15]:
pd.DataFrame(manual_results, index=[0])

,RMSE,MAE,R2,Stability
0,44.795695,15.571981,-32.245542,-0.001084


# Iterative Training

In [6]:
datasets['baselines']['full']['region_holdout']['europe_n_central_asia']['train'].shape

(275, 82)

In [7]:
datasets['baselines']['full']['region_holdout']['europe_n_central_asia'].keys()

dict_keys(['train', 'test', 'train_size', 'test_size'])

In [8]:
results = []

In [9]:
%%time
for scenario_type, scenario_dict in datasets.items():        # e.g., 'baselines', 'diversified'
    for scenario_name, splits in scenario_dict.items():      # e.g., 'full', 'constrained', 'max_balanced', etc.
        for split_type, data_group in splits.items():        # 'within_sample', 'region_holdout', 'size_holdout'
            if split_type == 'original_data':
                continue  # Skip original data entries
            
            # if split_type == 'region_holdout':
            #     continue # TEMP TO CHECK SMTH

            # For within-sample, we have one group; for others, multiple contexts
            # contexts = {'within_sample': data_group} if split_type == 'within_sample' else data_group

            # for context_name, data in contexts.items():
            #     print(f"Scenario: {scenario_type}, {scenario_name}, Split: {split_type}, Context: {context_name}")

            # Handle different split types
            if split_type == 'within_sample':
                # For within_sample, data structure is one level deeper
                data = data_group['within_sample']
                print(f"Scenario: {scenario_type}, {scenario_name}, Split: {split_type}")
                if 'train' not in data or 'test' not in data:
                    continue
                    
                train_df = data['train']
                test_df = data['test']

                print(train_df.shape, test_df.shape)

                # Separate features and target
                X_train = train_df.drop(columns=['totalEsg'])
                y_train = train_df['totalEsg']
                X_test  = test_df.drop(columns=['totalEsg'])
                y_test  = test_df['totalEsg']
                
                # Train and evaluate models
                for model_name, model in models.items():
                    # train
                    model.fit(X_train, y_train)

                    # predict
                    y_pred = model.predict(X_test)

                    # compute metrics
                    rmse, mae, r2 = compute_metrics(y_test, y_pred)
                    stability = compute_stability(model, X_test, y_test,
                                                noise_level=0.01, n_trials=5)
                    
                    results.append({
                        'scenario_type': scenario_type,
                        'scenario': scenario_name,
                        'split': split_type,
                        'context': 'within_sample',
                        'model': model_name,
                        'RMSE': rmse,
                        'MAE': mae,
                        'R2': r2,
                        'Stability': stability
                    })
            
            else:
                # For region_holdout and size_holdout, process each context
                for context_name, data in data_group.items():
                    print(f"Scenario: {scenario_type}, {scenario_name}, Split: {split_type}, Context: {context_name}")
                    if 'train' not in data or 'test' not in data:
                        continue
                        
                    train_df = data['train']
                    test_df = data['test']

                    print(train_df.shape, test_df.shape)

                    # Separate features and target
                    X_train = train_df.drop(columns=['totalEsg'])
                    y_train = train_df['totalEsg']
                    X_test  = test_df.drop(columns=['totalEsg'])
                    y_test  = test_df['totalEsg']
                    
                    # Train and evaluate models
                    for model_name, model in models.items():
                        # train
                        model.fit(X_train, y_train)

                        # predict
                        y_pred = model.predict(X_test)

                        # compute metrics
                        rmse, mae, r2 = compute_metrics(y_test, y_pred)
                        stability = compute_stability(model, X_test, y_test,
                                                  noise_level=0.01, n_trials=5)
                        
                        results.append({
                            'scenario_type': scenario_type,
                            'scenario': scenario_name,
                            'split': split_type,
                            'context': context_name,
                            'model': model_name,
                            'RMSE': rmse,
                            'MAE': mae,
                            'R2': r2,
                            'Stability': stability
                        })

results_df = pd.DataFrame(results)

Scenario: baselines, full, Split: within_sample
(291, 83) (73, 83)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=5.17923e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:42:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Scenario: baselines, full, Split: region_holdout, Context: east_asia_n_pacific
(301, 82) (63, 82)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=3.05446e-25): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:42:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Scenario: baselines, full, Split: region_holdout, Context: europe_n_central_asia
(275, 82) (89, 82)


...

Scenario: diversified, median_balanced, Split: size_holdout, Context: Mid-Cap
(275, 83) (125, 83)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.566e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:42:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Scenario: diversified, median_balanced, Split: size_holdout, Context: Small-Cap
(275, 83) (125, 83)


l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\sklearn\linear_model\_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=2.63994e-28): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
l:\Programs\anaconda\envs\esg_pred\Lib\site-packages\xgboost\training.py:183: UserWarning: [21:42:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


CPU times: total: 3min 40s
Wall time: 55.7 s


In [12]:
results_df = pd.DataFrame(results)

In [13]:
results_df

,scenario_type,scenario,split,context,model,RMSE,MAE,R2,Stability
0,baselines,full,within_sample,within_sample,Ridge,4.479570e+01,1.557198e+01,-3.224554e+01,-0.000533
1,baselines,full,within_sample,within_sample,RandomForest,6.850229e+00,5.258253e+00,2.225525e-01,0.033441
2,baselines,full,within_sample,within_sample,XGBoost,6.895373e+00,4.742770e+00,2.122718e-01,0.113209
3,baselines,full,within_sample,within_sample,MLP,2.817183e+08,4.534438e+07,-1.314895e+15,0.000160
4,baselines,full,region_holdout,east_asia_n_pacific,Ridge,1.710378e+02,7.587381e+01,-5.145711e+02,0.000472
...,...,...,...,...,...,...,...,...,...
135,diversified,median_balanced,size_holdout,Mid-Cap,MLP,7.898050e+07,1.222599e+07,-1.149300e+14,0.000269
136,diversified,median_balanced,size_holdout,Small-Cap,Ridge,2.045602e+01,1.292935e+01,-4.228887e+00,0.000406
137,diversified,median_balanced,size_holdout,Small-Cap,RandomForest,9.012373e+00,6.568490e+00,-1.495128e-02,-0.012209
138,diversified,median_balanced,size_holdout,Small-Cap,XGBoost,9.328139e+00,6.774992e+00,-8.731898e-02,-0.015248


In [14]:
results_df['R2'].max()

0.8538931691535758